In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import urllib.request
import mecab
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
df=pd.read_csv('./lable18_22.csv')
df.head()

,labels,kor_sentence,tokenized
0,1.0,[ET투자뉴스]CMG제약_기관의 힘? 대량순매수 이후.. 현재 +3.05%,"['[', 'ET', '투자', '뉴스', ']', 'CMG', '제약', '_',..."
1,1.0,[한경로보뉴스] '와이지엔터테인먼트' 52주 신고가 경신,"['[', '한경', '로보', '뉴스', ']', ""'"", '와', '이지', '..."
2,1.0,"CMG제약(058820) 종목알파고 분석, 외국인/기관 실시간 수급과 추가 매수 확률은?","['CMG', '제약', '(', '058820', ')', '종목', '알파', ..."
3,0.0,"CMG제약, 전일 대비 약 -4% 하락한 4,515원","['CMG', '제약', ',', '전일', '대비', '약', '-', '4', ..."
4,1.0,[Hot Stock] CMG제약,"['[', 'Hot', 'Stock', ']', 'CMG', '제약']"


In [4]:
stop_word = pd.read_csv('./stop_word_0.csv',names = ['idx','word'])
stop_word = stop_word.dropna()
# stop_word = stop_word.set_index('idx')
stop_word = stop_word.drop('idx',axis=1)

stop_word.head()

,word
1,으로
2,는
3,되
4,를
5,수


In [5]:
stop_word.iloc[0]

word    으로
Name: 1, dtype: object

In [6]:
stop_word.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1416 entries, 1 to 1416
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    1416 non-null   object
dtypes: object(1)
memory usage: 22.1+ KB


In [7]:
len(stop_word)

1416

In [8]:
# for i,word in enumerate(stop_word['word']):
#     print(i+1, word)
#     # print(word)

In [9]:
del_idx = [16,74,87,90,91,97,101,103,110,117,121,124,125,127,129,140,160,170,171,184,185,190,192,202,205,221,257,283,342,357
           ,372,397,408,430,438,460,513,549,577 ,581 ,589 ,592 ,605 ,613,621 ,638 ,671 ,680 ,694 ,699 ,712 ,725 ,752 ,756 ,777 
           ,780 ,785 ,799 ,825 ,828 ,845 ,849 ,860 ,873 ,887 ,893 ,894 ,899 ,918 ,929 ,935 ,939 ,946 ,960 ,964 ,971 ,979 ,984 ,989 ,1009
           ,1010 ,1011 ,1012 ,1014 ,1016 ,1018 ,1019 ,1021 ,1022 ,1026 ,1027 ,1030 ,1038 ,1053 ,1057 ,1058 ,1061 ,1063 ,1064 ,1066 ,1073 
           ,1074 ,1082 ,1092 ,1096 ,1106 ,1111 ,1114 ,1132 ,1137 ,1142 ,1165 ,1167 ,1182 ,1208 ,1252 ,1260 ,1262 ,1268 ,1272 ,1274 ,1281 
           ,1288 ,1294 ,1306 ,1327 ,1345 ,1356 ,1367 ,1374 ,1377 ,1382 ,1409]

In [10]:
stop_word.index

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1407, 1408, 1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416],
           dtype='int64', length=1416)

In [11]:
for idx in del_idx:
    # print(stop_word.iloc[idx-1])
    stop_word = stop_word.drop(idx-1, axis = 0)

In [12]:
len(stop_word)

1283

In [14]:
stop_word

,word
1,으로
2,는
3,되
4,를
5,수
...,...
1412,代
1413,뇨
1414,꿰
1415,찬


In [16]:
특수기호 = ['[','\\','{','}','[',']','/','?','.',',',';',':','|',')','*','~','`','^','_','<','>','@','#','$','%','&','=','(','\'','\"']

In [18]:
dic = {'word':특수기호}
temp = pd.DataFrame(dic)

In [21]:
stop_word = pd.concat([stop_word, temp], ignore_index=True)

In [22]:
stop_word_1 = stop_word


In [23]:
stop_word_1.to_csv('./stop_word_1.csv', index=False)